In [127]:
%pip -q install google-genai
%pip install -q google-adk

In [128]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [129]:
from google import genai
import os
import time

client = genai.Client()

MODEL_ID = "gemini-2.5-flash"

print("Listando modelos suportados para generateContent:")
# Use GenerativeModel.list_supported_models() em vez de genai.list_models()
try:
    for m in genai.GenerativeModel.list_supported_models():
      # Acessa o nome pela propriedade .name
      print(m.name)
except Exception as e:
    print(f"Não foi possível listar os modelos. Verifique sua configuração da API Key e permissões: {e}")

from IPython.display import HTML, Markdown



Listando modelos suportados para generateContent:
Não foi possível listar os modelos. Verifique sua configuração da API Key e permissões: module 'google.genai' has no attribute 'GenerativeModel'


In [108]:
!pip install -q google-generativeai
!pip install agential

In [109]:
!pip install -q google-adk

In [110]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

In [111]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [112]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [113]:
# --- Agente 1:  --- #

def agente_buscador(Sindrome):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para encontrar informacoes sobre a sindrome que foi informada.
        Procure Pelo menos 10 fontes relevantes, com base na fidelidade das noticias.
        Quanto mais referencias online melhor, para ser o mais relevante possivel.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Sindrome: {Sindrome}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [118]:
# --- Agente 2: Organizador --- #

def agente_organizador(sindrome, lancamentos_buscados):
    organizador = Agent(
        name="agente_organizador",
        model="gemini-2.5-pro-preview-03-25",
        # Inserir as instruções do Agente Organizador #################################################
        instruction="""
        Você é um pesquisador cientifico, especialista em sindromes raras. Com base na pesquisa feita
        pelo buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar
        o pontos mais importantes da sindrome, verificar formas de amparo a familia existente,
        formas de tratamento paleativos, e perguntas que os pais possam fazer aos medicos para prevenir probemas comuns
        da sindrome. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        """,
        description="Agente que reuni e organiza a pesquisa",
        tools=[google_search]
    )

    entrada_do_agente_organizador = f"Sindrome:{sindrome}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    organizacao_da_pesquisa = call_agent(organizador, entrada_do_agente_organizador)
    return organizacao_da_pesquisa

In [117]:
# --- Agente 3:  --- #

def agente_acolhedor(sindrome, organizacao_da_pesquisa):
    acolhedor = Agent(
        name="agente_acolhedor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Psicólogo do Serviço Social virtual, especialista em transmitir
            informações sobre síndromes raras para famílias de forma acolhedora,
            humana e empática. Sua principal função é receber dados 'Agente organizador' e apresentá-los aos pais.
            Com base no na pesquisa sua tarefa é comunicar essa informação aos pais de maneira concisa, amorosa e encorajadora.
            Sua fala deve ser clara e gentil, focando em validar os sentimentos dos pais. Ao final, inclua uma breve mensagem de apoio,
            sempre reforçando a importância de os pais conversarem com a equipe médica da criança sobre todas as informações.
            """,
        description="Agente acolhedor dos pais com filho/filha com sindrome."
    )
    entrada_do_agente_acolhedor = f"Sindrome: {sindrome}\nOrganizacao_da_pesquisa: {organizacao_da_pesquisa}"
    # Executa o agente
    Acolhimento = call_agent(acolhedor, entrada_do_agente_acolhedor)
    return Acolhimento

In [119]:
# Célula 06: Orquestração dos Agentes

from IPython.display import display, Markdown
import textwrap

print("🚀 Iniciando o Sistema de Informação sobre Síndromes Raras com 3 Agentes 🚀")

# --- Obter a Síndrome do Usuário ---
sindrome = input("❓ Por favor, digite o nome da SÍNDROME rara sobre a qual você quer obter informações: ")

# Inserir lógica do sistema de agentes ################################################
if not sindrome:
    print("Você esqueceu de digitar o nome da síndrome!")
else:
    print(f"Maravilha! Vamos buscar e organizar informações sobre {sindrome}.")

    # --- Executa Agente 1: Buscador ---
    print("\n--- ⏳ Executando Agente 1 (Buscador)... ---")
    try:
        # Chama o agente_buscador com a síndrome informada
        lancamentos_buscados = agente_buscador(sindrome)
        print("\n--- ✅ Resultado do Agente 1 (Buscador) ---\n")
        display(to_markdown(lancamentos_buscados))
    except Exception as e:
        print(f"❌ Ocorreu um erro ao executar o Agente Buscador: {e}")
        lancamentos_buscados = None # Define como None para não quebrar o fluxo, mas sinaliza falha

    print("--------------------------------------------------------------")

    # --- Executa Agente 2: Organizador (se o Buscador teve sucesso) ---
    if lancamentos_buscados:
        print("\n--- ⏳ Executando Agente 2 (Organizador)... ---")
        try:
            # Chama o agente_organizador com a síndrome e os resultados do buscador
            organizacao_da_pesquisa = agente_organizador(sindrome, lancamentos_buscados)
            print("\n--- ✅ Resultado do Agente 2 (Organizador) ---\n")
            display(to_markdown(organizacao_da_pesquisa))
        except Exception as e:
            print(f"❌ Ocorreu um erro ao executar o Agente Organizador: {e}")
            organizacao_da_pesquisa = None # Define como None para não quebrar o fluxo

        print("--------------------------------------------------------------")
    else:
        print("\n--- ⏭️ Pulando Agente 2 (Organizador) devido à falha no Agente Buscador. ---")
        organizacao_da_pesquisa = None


    # --- Executa Agente 3: Acolhedor (se o Organizador teve sucesso) ---
    if organizacao_da_pesquisa:
        print("\n--- ⏳ Executando Agente 3 (Acolhedor)... ---")
        try:
            # Chama o agente_acolhedor com a síndrome e os resultados organizados
            Acolhimento = agente_acolhedor(sindrome, organizacao_da_pesquisa)
            print("\n--- ✅ Resultado Final do Sistema (Agente 3 - Acolhedor) ---\n")
            display(to_markdown(Acolhimento))
        except Exception as e:
             print(f"❌ Ocorreu um erro ao executar o Agente Acolhedor: {e}")
             Acolhimento = "Não foi possível gerar a mensagem de acolhimento devido a um erro." # Mensagem de fallback
    else:
        print("\n--- ⏭️ Pulando Agente 3 (Acolhedor) devido à falha no Agente Organizador. ---")
        Acolhimento = "Não foi possível gerar a mensagem de acolhimento pois as etapas anteriores falharam." # Mensagem de fallback
        display(to_markdown(Acolhimento)) # Exibe a mensagem de fallback

    print("--------------------------------------------------------------")
    print("✨ Fluxo de agentes concluído. ✨")

🚀 Iniciando o Sistema de Informação sobre Síndromes Raras com 3 Agentes 🚀
❓ Por favor, digite o nome da SÍNDROME rara sobre a qual você quer obter informações: calibre


Exception in thread Thread-18 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u

Maravilha! Vamos buscar e organizar informações sobre calibre.

--- ⏳ Executando Agente 1 (Buscador)... ---

--- ✅ Resultado do Agente 1 (Buscador) ---



--------------------------------------------------------------

--- ⏭️ Pulando Agente 2 (Organizador) devido à falha no Agente Buscador. ---

--- ⏭️ Pulando Agente 3 (Acolhedor) devido à falha no Agente Organizador. ---


> Não foi possível gerar a mensagem de acolhimento pois as etapas anteriores falharam.

--------------------------------------------------------------
✨ Fluxo de agentes concluído. ✨
